In [ ]:
import rasterio
import rasterio.warp as rwarp

In [7]:
from osgeo import gdal
from osgeo import osr
import utm

In [ ]:
pdflayers

In [2]:
src = gdal.Open('NY_Mohonk_Lake_20190916_TM_geo.pdf', gdal.GA_ReadOnly)

In [3]:
type(src)

osgeo.gdal.Dataset

In [4]:
a = src.GetLayerCount()
a

0

In [ ]:
src.GetMetadata_List()

In [ ]:
src = gdal.Open('warped.tif')
src.GetLayerCount()

In [ ]:
res = gdal.Warp('warped.tif', src, dstSRS='EPSG:4087')
res.FlushCache()

In [ ]:
src

In [26]:
driver = gdal.GetDriverByName("GTiff")
dst = driver.Create('test.tif', 1024, 1024, 3, gdal.GDT_Byte)
srs = osr.SpatialReference()
srs.SetWellKnownGeogCS( 'EPSG:4087' )
a = src.GetGeoTransform()
src.SetGeoTransform((a[0], a[1], 0, a[3], 0, a[5]))
dst.GetGeoTransform()

(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)

In [ ]:
utm.from_latlon(41.875, -74.25)

In [23]:
gdal.ReprojectImage(src, dst)
dst.FlushCache()

In [ ]:
src.

In [27]:
ulx, uly, _, _ = utm.from_latlon(41.875, -74.25)
lrx, lry, _, _ = utm.from_latlon(41.75, -74.125)

a = gdal.Translate('test.tif', 'NY_Mohonk_Lake_20190916_TM_geo.pdf',
 resampleAlg='bilinear', projWin=(ulx,uly,lrx,lry))

ERROR 1: The -projwin option was used, but the geotransform is
rotated.  This configuration is not supported.


In [ ]:
src.GetGeoTransform()

In [ ]:
dst_crs = 'epsg:4087'
src = rasterio.open('~/Downloads/NY_Mohonk_Lake_20190916_TM_geo.pdf')


transform, width, height = rwarp.calculate_default_transform(
    src.crs, dst_crs, src.width, src.height, *src.bounds
)

kwargs = src.meta.copy()
kwargs.update({'crs': dst_crs, 'transform': transform, 'width': width, 'height': height})

with rasterio.open('junk.tif', 'w', **kwargs) as dst:
    for i in range(1, src.count + 1):
        rwarp.reproject(
            source=rasterio.band(src, i),
            destination=rasterio.band(dst, i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=rwarp.Resampling.bilinear)